# Import libraries

In [6]:
## Import libraries for machine learning and data processing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [7]:
import sys, importlib, site
print("sys.executable:", sys.executable)
print("sys.version:", sys.version)
print("sys.path (first 8):", sys.path[:8])
print("site.getsitepackages():", getattr(site, 'getsitepackages', lambda: None)())
print("USER site:", site.USER_SITE)
print("find tensorflow spec:", importlib.util.find_spec('tensorflow'))

sys.executable: /Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/tfpy/bin/python
sys.version: 3.11.14 (main, Oct  9 2025, 16:16:55) [Clang 17.0.0 (clang-1700.4.4.1)]
sys.path (first 8): ['/opt/homebrew/Cellar/python@3.11/3.11.14_1/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.14_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.14_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/tfpy/lib/python3.11/site-packages', '/Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/tfpy/lib/python3.11/site-packages/setuptools/_vendor']
site.getsitepackages(): ['/Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/tfpy/lib/python3.11/site-packages']
USER site: /Users/adityaponnada/Library/Python/3.11/lib/pyt

# Read data

In [8]:
## import dataset
raw_feature_df_scaled = pd.read_csv('/Users/adityaponnada/Downloads/time_study_data/processed_features_v100.csv')
## Display the first few rows of the dataset
raw_feature_df_scaled.head(5)

,participant_id,outcome,is_weekend,in_battery_saver_mode,charging_status,screen_on,dist_from_home,is_phone_locked,last_phone_usage,closeness_to_sleep_time,...,wake_day_part_24.0,wake_day_part_25.0,wake_day_part_26.0,wake_day_part_27.0,wake_day_part_28.0,wake_day_part_29.0,wake_day_part_30.0,wake_day_part_31.0,wake_day_part_32.0,wake_day_part_33.0
0,arrivejanitoruniformly@timestudy_com,1,1,NaN,NaN,0,0.000002,0.0,0.0,0.187031,...,0,0,0,0,0,0,0,0,0,0
1,arrivejanitoruniformly@timestudy_com,1,1,0.0,0.0,0,0.000002,0.0,0.0,0.180555,...,0,0,0,0,0,0,0,0,0,0
2,arrivejanitoruniformly@timestudy_com,1,1,NaN,NaN,0,0.000002,0.0,0.0,0.176050,...,0,0,0,0,0,0,0,0,0,0
3,arrivejanitoruniformly@timestudy_com,1,1,NaN,NaN,0,0.000002,0.0,0.0,0.172060,...,0,0,0,0,0,0,0,0,0,0
4,arrivejanitoruniformly@timestudy_com,1,1,NaN,NaN,0,0.000002,0.0,0.0,0.104721,...,0,0,0,0,0,0,0,0,0,0


# Missingness indicator

In [9]:
def add_missingness_indicators(df, id_col='participant_id', outcome_col='outcome', prefix='mi_'):
    """
    Add binary missingness indicator columns to `df` for every column except the id_col and outcome_col.

    Rules:
      - Skip columns named by id_col and outcome_col.
      - For a column named X, create column named prefix + X (e.g., 'mi_X').
      - Indicator is 1 when the original value is NaN, else 0.

    The function returns the DataFrame with the new indicator columns added (inplace on a copy).
    """
    import pandas as pd
    if df is None or not hasattr(df, 'copy'):
        raise ValueError('df must be a pandas DataFrame')

    result = df if df is None else df.copy()
    skip = {id_col, outcome_col}
    for col in result.columns.tolist():
        if col in skip:
            continue
        # skip already-indicator columns to avoid creating mi_mi_X
        if str(col).startswith(prefix):
            continue
        mi_col = f'{prefix}{col}'
        # compute indicator: 1 if NaN, else 0. Use isna for pandas types
        try:
            result[mi_col] = result[col].isna().astype(int)
        except Exception:
            # fallback: use pandas isnull
            result[mi_col] = pd.isnull(result[col]).astype(int)
    return result

# Example usage:
raw_feature_df_scaled = add_missingness_indicators(raw_feature_df_scaled)
raw_feature_df_scaled[['in_battery_saver_mode', 'mi_in_battery_saver_mode']].head()


,in_battery_saver_mode,mi_in_battery_saver_mode
0,NaN,1
1,0.0,0
2,NaN,1
3,NaN,1
4,NaN,1


# Split training and test

In [10]:
def split_train_test_by_users_random(df, id_col='participant_id', n_train_users=10, random_state=None):
    """
    Randomly split a DataFrame into a train set containing all rows for a randomly
    selected set of `n_train_users` participants and a test set containing the
    remaining participants.

    Returns: (train_df, test_df) with indices reset.
    """
    import numpy as np
    import pandas as pd

    if id_col not in df.columns:
        raise ValueError(f"id_col '{id_col}' not found in DataFrame columns")

    unique_ids = pd.Index(df[id_col].dropna().unique())
    n_unique = len(unique_ids)
    if n_unique == 0:
        raise ValueError('No participant ids found in the DataFrame')
    if n_train_users <= 0 or n_train_users >= n_unique:
        raise ValueError(f'n_train_users must be >0 and < number of unique participants ({n_unique})')

    rng = np.random.default_rng(random_state)
    train_ids = rng.choice(unique_ids, size=n_train_users, replace=False)

    train_df = df[df[id_col].isin(train_ids)].reset_index(drop=True)
    test_df = df[~df[id_col].isin(train_ids)].reset_index(drop=True)

    return train_df, test_df

# Example usage:
train_df, test_df = split_train_test_by_users_random(raw_feature_df_scaled, n_train_users=10, random_state=42)

# Missing data imputation

For features like location, we will use median imputation. For other features we will forward the last known data (using Linear Interpolation)

In [11]:
# Print the percentage of missing values for each column in train_df
missing_pct_train = train_df.isnull().mean() * 100
print("% Missing values per column in train_df:")
missing_pct_train.sort_values(ascending=False)

% Missing values per column in train_df:


in_battery_saver_mode    52.497249
charging_status          52.497249
dist_from_home           12.897827
mims_5min                 5.864054
is_phone_locked           1.442592
                           ...    
wake_day_part_12.0        0.000000
wake_day_part_11.0        0.000000
wake_day_part_10.0        0.000000
wake_day_part_9.0         0.000000
mi_wake_day_part_33.0     0.000000
Length: 122, dtype: float64

In [12]:
def missing_values_table(train_df, show=True):
    """
    Compute a DataFrame summarizing missing data per column for `train_df` and optionally print it in full.

    Returns a pandas DataFrame with columns: 'missing_count' and 'missing_pct' (0-100), sorted by missing_pct desc.

    Parameters:
    - train_df: pandas DataFrame to analyze
    - show: if True, print the full DataFrame without truncation
    """
    import pandas as pd

    if train_df is None or not hasattr(train_df, 'isnull'):
        raise ValueError('train_df must be a valid pandas DataFrame')

    n_rows = len(train_df)
    missing_count = train_df.isnull().sum()
    missing_pct = (missing_count / n_rows) * 100 if n_rows > 0 else missing_count * 0.0
    df_missing = pd.DataFrame({'missing_count': missing_count, 'missing_pct': missing_pct})
    df_missing = df_missing.sort_values('missing_pct', ascending=False)

    if show:
        # display entire table without truncation
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
            print(df_missing)

    return df_missing

# Example usage:
# df_missing = missing_values_table(train_df)
# or to get the Series of percentages: df_missing['missing_pct']


## Linear interpolation based impuation

In [13]:
def interpolate_impute(train_df, cols=None, method='linear', limit_direction='both', axis=0, inplace=False, fill_remaining_with='median'):
    """
    Perform linear-interpolation based imputation on `train_df` for the specified columns.

    - train_df: pandas DataFrame
    - cols: list of columns to interpolate. If None, numeric columns will be used.
    - method: interpolation method passed to pandas.DataFrame.interpolate (default 'linear').
    - limit_direction: passed to interpolate (default 'both').
    - axis: axis for interpolation (0 for index, 1 for columns).
    - inplace: if True, modify train_df in place and return it; otherwise return a copy.
    - fill_remaining_with: if 'median', fill remaining NaNs with column median after interpolation;
      if 'ffill', use forward-fill then backward-fill; if None, leave NaNs as-is.

    Returns the imputed DataFrame.
    """
    import pandas as pd
    import numpy as np

    if train_df is None or not hasattr(train_df, 'copy'):
        raise ValueError('train_df must be a valid pandas DataFrame')

    df = train_df if inplace else train_df.copy()

    if cols is None:
        # default: numeric columns only
        cols = df.select_dtypes(include=[np.number]).columns.tolist()
    else:
        # ensure requested cols exist
        cols = [c for c in cols if c in df.columns]

    if not cols:
        # nothing to do
        return df

    # Interpolate numeric columns
    try:
        df[cols] = df[cols].interpolate(method=method, limit_direction=limit_direction, axis=axis)
    except Exception as e:
        print(f'Interpolation failed: {e}; returning original or partially-imputed DataFrame')

    # Optionally fill remaining NaNs
    if fill_remaining_with == 'median':
        for c in cols:
            if df[c].isna().any():
                try:
                    med = df[c].median()
                    df[c].fillna(med, inplace=True)
                except Exception:
                    pass
    elif fill_remaining_with == 'ffill':
        df[cols] = df[cols].ffill().bfill()
    # if None, leave remaining NaNs as-is

    return df

# Example usage:
train_df = interpolate_impute(train_df, cols=None, method='linear', inplace=True)  # numeric cols only
# To operate in-place: interpolate_impute(train_df, inplace=True)


In [14]:
train_df.head(5)

,participant_id,outcome,is_weekend,in_battery_saver_mode,charging_status,screen_on,dist_from_home,is_phone_locked,last_phone_usage,closeness_to_sleep_time,...,mi_wake_day_part_24.0,mi_wake_day_part_25.0,mi_wake_day_part_26.0,mi_wake_day_part_27.0,mi_wake_day_part_28.0,mi_wake_day_part_29.0,mi_wake_day_part_30.0,mi_wake_day_part_31.0,mi_wake_day_part_32.0,mi_wake_day_part_33.0
0,catsupexploitmocker@timestudy_com,0,0,0.0,1.0,0,0.000003,1.0,0.063333,0.923111,...,0,0,0,0,0,0,0,0,0,0
1,catsupexploitmocker@timestudy_com,0,0,0.0,1.0,0,0.000053,1.0,0.366667,0.906962,...,0,0,0,0,0,0,0,0,0,0
2,catsupexploitmocker@timestudy_com,0,0,0.0,1.0,0,0.000005,1.0,0.518333,0.898969,...,0,0,0,0,0,0,0,0,0,0
3,catsupexploitmocker@timestudy_com,0,0,0.0,1.0,1,0.000005,0.0,0.000000,0.891005,...,0,0,0,0,0,0,0,0,0,0
4,catsupexploitmocker@timestudy_com,0,0,0.0,1.0,0,0.000006,1.0,0.016667,0.883027,...,0,0,0,0,0,0,0,0,0,0


## RNN Training

In [15]:
train_df.shape[0]
train_df.shape[1]

122

In [16]:
## Number of observations per participant
# Compute and print the number of observations per participant_id in train_df
try:
    counts = train_df['participant_id'].value_counts().sort_index()
    print('Number of observations per participant_id:')
    print(counts.to_string())
    # store the maximum count in obs_len
    obs_len = int(counts.max()) if not counts.empty else 0
    print(f'obs_len (max observations per participant) = {obs_len}')
except NameError:
    print('train_df is not defined. Run the split to create train_df first.')
    obs_len = None
except Exception as e:
    print('Error computing observation counts:', e)
    obs_len = None


Number of observations per participant_id:
participant_id
catsupexploitmocker@timestudy_com          14639
certifiedembargobartender@timestudy_com    11555
diagramuncoupleoutput@timestudy_com        11802
kangaroozodiaccrudeness@timestudy_com       9143
predatordebatingpredator@timestudy_com     12507
residentselfgutter@timestudy_com           14338
retrievergeckoabroad@timestudy_com            55
superiorpassablecosmic@timestudy_com       11674
urchinvariablytrend@timestudy_com          15868
whoeverrelightspookily@timestudy_com        8360
obs_len (max observations per participant) = 15868


In [17]:
num_users = train_df['participant_id'].nunique()
print(num_users)

10


In [18]:
print(train_df.columns)


Index(['participant_id', 'outcome', 'is_weekend', 'in_battery_saver_mode',
       'charging_status', 'screen_on', 'dist_from_home', 'is_phone_locked',
       'last_phone_usage', 'closeness_to_sleep_time',
       ...
       'mi_wake_day_part_24.0', 'mi_wake_day_part_25.0',
       'mi_wake_day_part_26.0', 'mi_wake_day_part_27.0',
       'mi_wake_day_part_28.0', 'mi_wake_day_part_29.0',
       'mi_wake_day_part_30.0', 'mi_wake_day_part_31.0',
       'mi_wake_day_part_32.0', 'mi_wake_day_part_33.0'],
      dtype='object', length=122)


In [19]:
## Print the shape of train_df and count feature columns excluding id/outcome
try:
    n_cols_total = train_df.shape[1]
    print('Total columns in train_df:', n_cols_total)
    # define which columns to exclude from feature count
    exclude_cols = ['participant_id', 'outcome']
    feature_cols = [c for c in train_df.columns if c not in exclude_cols]
    n_feature_cols = len(feature_cols)
    print(f'Number of columns excluding {exclude_cols}: {n_feature_cols}')
except NameError:
    print('train_df is not defined. Run the split to create train_df first.')
    n_feature_cols = None
except Exception as e:
    print('Error computing column counts:', e)
    n_feature_cols = None


Total columns in train_df: 122
Number of columns excluding ['participant_id', 'outcome']: 120


In [21]:
# Pad sequences for RNN input by building sequences directly from `train_df` / `test_df`.
# Assumes `train_df` and optionally `test_df` exist and are already scaled.
# We group rows by `participant_id`, collect feature columns (exclude id/outcome),
# optionally sort by a timestamp column if present, then pad to obs_len (or inferred).

from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

PADDING_VALUE = -999.0
ID_COL = 'participant_id'
OUTCOME_COL = 'outcome'

# Verify train_df exists
if 'train_df' not in globals() or train_df is None:
    raise RuntimeError('train_df not found. Run the split/preprocessing cells before padding.')

# Choose feature columns: all except ID_COL and OUTCOME_COL
exclude_cols = {ID_COL, OUTCOME_COL}
feature_cols = [c for c in train_df.columns if c not in exclude_cols]
if not feature_cols:
    raise RuntimeError(f'No feature columns found after excluding {exclude_cols}')

# If there's a timestamp-like column, use it to sort within each participant
timestamp_candidates = ['timestamp','time','start_time','started_at','created_at','event_time']
sort_col = next((c for c in timestamp_candidates if c in train_df.columns), None)

def _build_sequences_from_df(df):
    """Return a list of 2D numpy arrays (timesteps x features), one per participant in df."""
    seqs = []
    # group preserves order only if df is already ordered; we'll sort if a sort_col exists
    if sort_col is not None:
        grouped = df.sort_values(sort_col).groupby(ID_COL, sort=True)
    else:
        grouped = df.groupby(ID_COL, sort=True)

    for pid, group in grouped:
        # extract feature columns as a 2D array
        arr = group[feature_cols].to_numpy(dtype=np.float32)
        seqs.append(arr)
    return seqs

# Build train sequences
X_train_seqs = _build_sequences_from_df(train_df)
print(f'Built {len(X_train_seqs)} training sequences. Example lengths (first 5):', [len(s) for s in X_train_seqs[:5]])

# Build test sequences if test_df exists
X_test_seqs = None
if 'test_df' in globals() and test_df is not None:
    X_test_seqs = _build_sequences_from_df(test_df)
    print(f'Built {len(X_test_seqs)} test sequences. Example lengths (first 5):', [len(s) for s in X_test_seqs[:5]])

# Determine MAX_LENGTH: prefer obs_len, else infer from sequences
if 'obs_len' in globals() and obs_len is not None and int(obs_len) > 0:
    MAX_LENGTH = int(obs_len)
else:
    all_lengths = [len(s) for s in X_train_seqs]
    if X_test_seqs is not None:
        all_lengths += [len(s) for s in X_test_seqs]
    if not all_lengths:
        raise RuntimeError('No sequences found to infer MAX_LENGTH')
    MAX_LENGTH = int(max(all_lengths))

print(f'Using MAX_LENGTH = {MAX_LENGTH} for padding')

# Pad sequences into 3D arrays (n_users, time_steps, n_features)
try:
    X_train_padded = pad_sequences(X_train_seqs, maxlen=MAX_LENGTH, dtype='float32', padding='post', truncating='post', value=PADDING_VALUE)
    print('X_train_padded shape:', X_train_padded.shape)
except Exception as e:
    X_train_padded = None
    print('Failed to pad training sequences:', e)

X_test_padded = None
if X_test_seqs is not None:
    try:
        X_test_padded = pad_sequences(X_test_seqs, maxlen=MAX_LENGTH, dtype='float32', padding='post', truncating='post', value=PADDING_VALUE)
        print('X_test_padded shape:', X_test_padded.shape)
    except Exception as e:
        X_test_padded = None
        print('Failed to pad test sequences:', e)

# Create boolean masks (True = valid timestep)
train_mask = None
test_mask = None
if X_train_padded is not None:
    train_mask = np.any(X_train_padded != PADDING_VALUE, axis=-1)
    print('train_mask shape:', train_mask.shape)
if X_test_padded is not None:
    test_mask = np.any(X_test_padded != PADDING_VALUE, axis=-1)
    print('test_mask shape:', test_mask.shape)

# Expose useful names to the notebook globals
# X_train_seqs, X_test_seqs, X_train_padded, X_test_padded, train_mask, test_mask, feature_cols, MAX_LENGTH


Built 10 training sequences. Example lengths (first 5): [14639, 11555, 11802, 9143, 12507]
Built 90 test sequences. Example lengths (first 5): [10845, 13238, 12257, 12757, 4692]
Using MAX_LENGTH = 15868 for padding
X_train_padded shape: (10, 15868, 120)
Built 90 test sequences. Example lengths (first 5): [10845, 13238, 12257, 12757, 4692]
Using MAX_LENGTH = 15868 for padding
X_train_padded shape: (10, 15868, 120)
X_test_padded shape: (90, 15868, 120)
train_mask shape: (10, 15868)
test_mask shape: (90, 15868)
X_test_padded shape: (90, 15868, 120)
train_mask shape: (10, 15868)
test_mask shape: (90, 15868)
